<a href="https://colab.research.google.com/github/vfrantc/dehaze_detect/blob/main/evaluate_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!nvidia-smi

Mon Nov  8 09:51:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


# RUN DETECTOR ON UNPROCESSED IMAGES

In [3]:
!pip install mmcv-full
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -e .
!pip install Pillow==7.0.0

     |████████████████████████████████| 389 kB 13.1 MB/s 
     |████████████████████████████████| 185 kB 67.4 MB/s 
  Created wheel for mmcv-full: filename=mmcv_full-1.3.16-cp37-cp37m-linux_x86_64.whl size=43446760 sha256=8aead6dd55754e4cb5eef91f2a1f95da1758ad2853b94212e0fd366b11f8d90d
  Stored in directory: /root/.cache/pip/wheels/ea/0a/1e/95428dfef725d66e6693b61180b928c293489b6d13df3ae0e2
Successfully built mmcv-full
Cloning into 'mmdetection'...
remote: Enumerating objects: 21556, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 21556 (delta 1), reused 3 (delta 1), pack-reused 21550
Receiving objects: 100% (21556/21556), 25.07 MiB | 21.25 MiB/s, done.
Resolving deltas: 100% (15129/15129), done.
/content/mmdetection
Obtaining file:///content/mmdetection
  Created wheel for terminaltables: filename=terminaltables-3.1.0-py3-none-any.whl size=15354 sha256=386b5674f5c817691a35ec11f5141374400d7913e49ad3de1a6c452c98288938
  Stor

In [4]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmdetection/v2.0/scnet/scnet_x101_64x4d_fpn_20e_coco/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth \
      -O checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth

--2021-11-08 08:59:43--  https://download.openmmlab.com/mmdetection/v2.0/scnet/scnet_x101_64x4d_fpn_20e_coco/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.254.186.246
Connecting to download.openmmlab.com (download.openmmlab.com)|47.254.186.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 611860651 (584M) [application/octet-stream]
Saving to: ‘checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth’

checkpoints/scnet_x 100%[===================>] 583.52M  11.5MB/s    in 55s     

2021-11-08 09:00:39 (10.7 MB/s) - ‘checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth’ saved [611860651/611860651]



In [6]:
%cd mmdetection

/content/mmdetection


In [7]:
from glob import glob
import os

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.datasets import build_dataloader
from mmdet.apis import single_gpu_test
from mmcv.parallel import MMDataParallel
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

config = 'configs/scnet/scnet_x101_64x4d_fpn_20e_coco.py'
checkpoint = 'checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth'
model = init_detector(config, checkpoint, device='cuda:0')
model = MMDataParallel(model, device_ids=[0])

Use load_from_local loader


In [8]:
def create_dataset(folder):
  cfg = Config.fromfile('./configs/scnet/scnet_x101_64x4d_fpn_20e_coco.py')
  cfg.dataset_type = 'CocoDataset'
  PREFIX = folder
  cfg.data.test.img_prefix = folder
  cfg.data.test.ann_file = os.path.join(PREFIX, 'new_test.json')
  cfg.data.test.type = 'CocoDataset'
  dataset = build_dataset(cfg.data.test)
  #dataset.CLASSES = classes
  #dataset.cat_ids = list(range(len(dataset.CLASSES)))
  data_loader = build_dataloader(dataset, samples_per_gpu=1, workers_per_gpu=2, dist=False, shuffle=False)
  return dataset, data_loader

In [8]:
!cp /content/drive/MyDrive/hazy_detect/out_gca.zip .
!cp /content/drive/MyDrive/hazy_detect/dcp_coco.zip . 
!cp /content/drive/MyDrive/hazy_detect/out_refined.tar.gz .
!cp /content/drive/MyDrive/hazy_detect/dehazenet_coco.zip .
!cp /content/drive/MyDrive/hazy_detect/ide_coco.zip .

In [9]:
!unzip -q out_gca.zip
!unzip -q dcp_coco.zip
!tar xzvf out_refined.tar.gz
!unzip -q dehazenet_coco.zip
!unzip -q ide_coco.zip

out_refined/
out_refined/new_test.json


### gca

In [9]:
!cp /content/drive/MyDrive/new_test.json out_gca

In [ ]:
method = 'out_gca'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
outputs = single_gpu_test(model, data_loader, True, f'./out_{method}', 0.3)


mkdir: cannot create directory ‘out_out_gca’: File exists
loading annotations into memory...
Done (t=0.27s)
creating index...
index created!
[                                                  ] 0/4320, elapsed: 0s, ETA:

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[>>>>>>>>>>>>>>>>>>        ] 3064/4320, 1.5 task/s, elapsed: 2088s, ETA:   856s

In [ ]:
# post-process the detection to make sure that we use only the correct ones

In [ ]:
dataset.cat_ids.append(91)
metric = dataset.evaluate(outputs, metric='bbox')

In [ ]:
!tar czf detected_gca.tar.gz out_out_gca/
!cp detected_gca.tar.gz /content/drive/MyDrive/new_detect

### dcp


In [ ]:
method = 'dcp_coco'
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, True, f'./out_{method}', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

In [ ]:
!tar czf detected_dcp.tar.gz out_dcp_coco/
!cp -r detected_dcp.tar.gz /content/drive/MyDrive/new_detect

### Refined

In [ ]:
!cp /content/drive/MyDrive/new_test.json out_refined

In [ ]:
!ls out_refined/JPEGImages

In [ ]:
method = 'out_refined'
!rm -rf out_out_refined
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, True, f'./out_{method}', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

In [ ]:
!tar czf detected_refined.tar.gz out_out_refined/
!cp -r detected_refined.tar.gz /content/drive/MyDrive/new_detect

# Dehazenet

In [ ]:
method = 'dehazenet_coco'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, True, f'./out_{method}', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

In [ ]:
!tar czf detected_dehazenet.tar out_dehazenet/
!cp -r detected_dehazenet.zip /content/drive/MyDrive/new_detect

# IDE

In [ ]:
method = 'dide_coco'
!cp /content/drive/MyDrive/new_test.json dide_coco
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, True, f'./out_{method}', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

In [ ]:
!tar czf detected_ide.tar.gz out_ide_coco/
!cp -r detected_ide.tar.gz /content/drive/MyDrive/new_detect

# RTTS

In [ ]:
!rm -rf rtts_coco
!cp /content/mmdetection/drive/MyDrive/haze/data/RTTS.zip .
!unzip -q RTTS.zip
!mv RTTS rtts_coco 
!cp /content/mmdetection/drive/MyDrive/new_test.json rtts_coco

In [ ]:
method = 'rtts_coco'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, True, f'./out_{method}', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

In [ ]:
!tar czf detected_rtts_coco.tar.gz out_rtts_coco/
!cp -r detected_rtts_coco.tar.gz /content/drive/MyDrive/new_detect